# This notebook deals with:
1. Using Gemini and YouTube API to fetch and get transcript and notes
2. Using ChromaDB to embed those
3. Using Gemini to get response to queries

In [1]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))

<IPython.core.display.Javascript object>

In [2]:
!pip3 install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.0 MB/s eta 0:0

In [4]:
!pip3 install youtube-transcript-api

In [5]:
!pip3 install google-generativeai

In [6]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
import google.generativeai as genai
import chromadb
from chromadb.utils import embedding_functions
import os

In [8]:
GEMINI_API_KEY = '<your_api_key>' # Get from https://ai.google.dev/gemini-api/docs/quickstart
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel('models/gemini-2.0-flash') # from https://ai.google.dev/gemini-api/docs/models#gemini-2.0-flash
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# ef = embedding function
gemini_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=GEMINI_API_KEY)

collection = chroma_client.get_or_create_collection(name='gaurav_yt_notes', embedding_function=gemini_ef)

In [11]:
yt_video_id = 'gYhY-k4DQvE' # https://youtu.be/gYhY-k4DQvE

transcript = YouTubeTranscriptApi.get_transcript(yt_video_id, languages=['en'])
# transcript = TextFormatter().format_transcript(transcript)

with open('yt_transcript.txt', 'w') as file:
  file.write(str(transcript))

In [13]:
prompt = 'Extract key notes from video transcript.'
response = model.generate_content(prompt + str(transcript))

with open('yt_transcript_notes.txt', 'w') as file:
  file.write(str(response))

In [14]:
with open('yt_transcript_notes.txt', 'r') as file:
  notes = file.read()

collection.upsert(
    documents=[notes],
    ids=[yt_video_id]
)

In [23]:
query = 'What is this video concerned with?'

results = collection.query(
    query_texts=[query],
    n_results=1,
    include=['documents', 'distances']
)

results['ids'][0]
results['documents'][0]

['response:\nGenerateContentResponse(\n    done=True,\n    iterator=None,\n    result=protos.GenerateContentResponse({\n      "candidates": [\n        {\n          "content": {\n            "parts": [\n              {\n                "text": "Here are the key notes from the video transcript:\\n\\n*   **Goal:** Generate high-quality notes from YouTube videos using Python code and save them to a vector database (Qdrant) for future reference.\\n*   **Packages Needed:**\\n    *   `youtube_transcript_api`: To extract the transcript from the YouTube video.\\n    *   Google Gemini (large language model): To convert the transcript into notes (free tier available).\\n    *   Qdrant: To save notes into a vector database.\\n*   **Gemini API Key:** Required to use the Gemini model.\\n*   **Model Code:** Needed to instantiate the Gemini model (available via a link).\\n*   **Qdrant Vector Database:** Used for storing the notes; initialized with `kb.persistent_client`. An embedding function (using G

In [22]:
query += str(results['documents'][0])

results = model.generate_content(query)

results.text

'The video is concerned with generating high-quality notes from YouTube videos using Python code, a large language model (Google Gemini), and a vector database (Qdrant). The process involves extracting the video transcript, using Gemini to convert it into notes, and saving those notes into Qdrant for future searching and retrieval.\n'

# by Gaurav Kabra